In [1]:
import pandas as pd
import re
from ast import literal_eval
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file, datas
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '90'
formatted_attribute = 'volume_oz'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'
field1='Capacity (Volume) [Nom]'
fields = [field1]

In [3]:
df, custom_field_df=parameters(customer_id, formatted_attribute, field1)
# [*custom_field_df]
df_cur, df_custom=datas(df,custom_field_df, field1)

253


In [4]:
del df_cur['bucket_id']
del df_cur['product_id']
del df_cur['image_url']
del df_cur['value']

In [4]:
# def rounding(heights,col,texto):
#     regex_pattern_rounding=rf'''(?i)(\d+\.\d\d\d)|(\.0(?!\d))|(\.\d0)|(xa)|(\\)|([{texto}])|(\')|(\d\d\d\d)|()'''         
#     heights['rounding'] = heights[col].apply(lambda x: re_extract(regex_pattern_rounding, x))
#     return heights[heights['rounding'].astype(str)!='[]']


In [11]:
df_custom=custom_field(df_custom,field1,formatted_attribute)
pat=r'''(?i)(oz)|()'''
df_custom['match'] = df_custom[curation_col].apply(lambda x: re_extract(pat, x))
df_custom[curation_col]=df_custom[curation_col].apply(lambda x: re.sub(r'(?i)(?<=oz)\.','',str(x)))
df_custom=df_custom[df_custom['match'].astype(str)!='[]']

matchcust=df_custom[['external_id',curation_col]]
print(len(matchcust))
rounding(df_custom,curation_col, 'a-np-y')

35


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, Capacity (Volume) [Nom], match, Q:volume_oz, rounding]
Index: []

In [12]:
df_custom[curation_col].explode().value_counts()

32 oz      6
0.24 oz    6
17 oz      4
20 oz      3
0.77 oz    3
180 oz     2
18 oz      2
70 oz      1
13 oz      1
0.18 oz    1
24 oz      1
0.17 oz    1
16 oz      1
0.3 oz     1
0.14 oz    1
4 oz       1
Name: Q:volume_oz, dtype: int64

In [11]:
# from natsort import natsorted
# regex_pattern=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}(?:\'|\'\'|\"|\”|inc?h?|fe?e?t|foot)[\W]{0,3}(?:Width|W)\b)|()'''           
# df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

# # Filter out the SKUs that dont have numbers
# pat_number=r'\d'
# df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(pat_number, str(x)))
# lengthNew=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]
# del lengthNew['new_matches']

# lengthNew['matches']=lengthNew['matches'].apply(lambda x: natsorted(x)).apply(lambda x: sorted(x))
# lengthNew['matchez']=lengthNew['matches'].apply(lambda x: re.sub(r'''(?i)((?:\\?\"|\”|inc?h?|'')?\W{0,2}(?:Width|W))|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'\\" W',' in',str(x))).apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"|�|•','',str(x))).apply(lambda x: re.sub(r'''(?i)(\\?(?:(?:(?<!')'(?!'))|(?:ft)|(?:foot)|(?:feet))\W?(Width|W))''',' ft',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))


# curate(lengthNew, 'matchez', formatted_attribute)
# print(len(lengthNew))
# matchesnew=lengthNew[['external_id',curation_col]]
# matchesnew[curation_col].explode().value_counts()
# rounding(lengthNew, 'matchez')

# N/A Values

In [9]:
df_na=df_cur
regex_pattern_na=r'''(?i)((?<!\d)(?<!\w)(?<!\/)(?<!-)\.?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\s?oz)|(\boz\b)|((?<!Use\s\d\-)ounce(?!.milkshake))|()'''
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

df_na=df_na[(df_na['na_matches'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))
df_na.head(5)

169


buckets external_id  \
4  Jams, Jellies, & Preserves  5637396196   
5  Jams, Jellies, & Preserves  5637400293   
6    Milk & Milk Alternatives  5637457500   
7    Milk & Milk Alternatives  5637457501   
8       Gum, Candies, & Mints  5637399785   

                                  product_name  \
4  Smucker's Strawberry Preserves, 4 lb., 6/PK   
5        Smucker's Strawberry Preserves, 4 lb.   
6               Oringer Powdered Malt, 2.5 lb.   
7         Oringer Powdered Malt, 2.5 lb., 4/CS   
8                    Skittles Skittles, 25 lb.   

                                                                                       long_desc  \
4  Strawberry Preserves, 4 lb., 6/PK•Versatile ingredients that can be incorporated into recipes   
5        Strawberry Preserves, 4 lb.•Versatile ingredients that can be incorporated into recipes   
6                        Powdered Malt, 2.5 lb.•Full flavored, free flowing and dissolves easily   
7                  Powdered Malt, 2.5 lb., 4/CS•Full flavored, free flowing and dissolves easily   
8                                 Skittles, 25 lb.•Great for baking, ice cream toppings and more   

                                                      url  \
4  https://wbmason.com/ProductDetail.aspx?ItemID=SMU00308   
5  https://wbmason.com/ProductDetail.aspx?ItemID=SMU00308   
6  https://wbmason.com/ProductDetail.aspx?ItemID=PDIDT383   
7  https://wbmason.com/ProductDetail.aspx?ItemID=PDIDT383   
8  https://wbmason.com/ProductDetail.aspx?ItemID=WMW06560   

  Capacity (Volume) [Nom] na_matches Q:volume_oz  
4                     NaN         []         n/a  
5                     NaN         []         n/a  
6                     NaN         []         n/a  
7                     NaN         []         n/a  
8                     NaN         []         n/a

In [13]:
# d = {'col1': ['["1 oz","2 oz","3 oz"]', '["1.2 in","1 in","1.3 in"]', '["10 in","22 in","3.4 in"]']}
# a = pd.DataFrame(data=d)
# a

In [14]:
# from natsort import natsorted

# a['col1'].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')

# a['col1'].apply(lambda x: natsorted(eval(x)))



# a['col1'].apply(lambda x: sorted(x))

# df_oz['match'].apply(lambda x : literal_eval(x)).apply(lambda x: sorted(x))

# df_oz
# df_oz['match'].sort(key=grp)

# df_oz['match'].apply(lambda y: sorted(y, key=lambda x:int(re.findall('\d+', x))))

# df_oz['match'].apply(lambda x: sorted_alphanumeric((x)))
# df_oz['match'].apply(lambda x: natsorted(x))

In [15]:
# y=["8.5 oz","16 oz","32 oz","8 oz","8.1 oz"]
# sorted_alphanumeric(y)

In [72]:
# def sorted_alphanumeric(data):
#     convert = lambda text: int(text) if text.isdigit() else text.lower()
#     alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
#     return sorted(data, key=alphanum_key)

In [16]:
# def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
#     return [int(text) if text.isdigit() else text.lower()
#             for text in _nsre.split(s)]

In [44]:
# natsort = lambda s: [int(t) if t.isdigit() else t.lower() for t in re.split('(\d+)', s)]
# # L = ["a1", "a10", "a11", "a2", "a22", "a3"]   
# # print(sorted(L, key=natsort))

In [55]:
# def grp(txt): 
#     s = re.search(r'Test-(\d+)\.model', txt, re.IGNORECASE)
#     if s:
#         return int(s.group(1))
#     else:
#         return float('-inf')  # Sorts non-matching strings ahead of matching strings

In [15]:
df_oz=df_cur[df_cur['na_matches'].astype(str)!='[]']
#Get the values in the form:["1 in","2","3"]
df_oz['na_matches']=df_oz['na_matches'].apply(lambda x: sorted(x)).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r'\,','","',str(x))).apply(lambda x: re.sub(r"-",' ',str(x))).apply(lambda x: re.sub(r'(?<!\s)oz',' oz',str(x))).apply(lambda x: re.sub(r'(?<!\d)\.','0.',str(x)))
curate(df_oz, 'na_matches', formatted_attribute, 'a-np-y')

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, Capacity (Volume) [Nom], na_matches, Q:volume_oz, rounding]
Index: []

In [16]:
matchoz=df_custom[['external_id',curation_col]]
print(len(matchoz))
rounding(matchoz,curation_col, 'a-np-y')

35


Empty DataFrame
Columns: [external_id, Q:volume_oz, rounding]
Index: []

In [17]:
df_custom

buckets  bucket_id  product_id  \
31                         Adhesives & Glues       8615   434388085   
32                         Adhesives & Glues       8615   434388086   
33                         Adhesives & Glues       8615   434388821   
34                         Adhesives & Glues       8615   434388822   
35                         Adhesives & Glues       8615   434389609   
41                         Adhesives & Glues       8615   434408534   
42                         Adhesives & Glues       8615   434408947   
43                         Adhesives & Glues       8615   434408948   
51                         Adhesives & Glues       8615   434439260   
52                         Adhesives & Glues       8615   434439261   
94                         Adhesives & Glues       8615   434483916   
96                         Adhesives & Glues       8615   434484127   
97                         Adhesives & Glues       8615   434484128   
98                         Adhesives & Glues       8615   434484137   
111  Safety Wash Station Parts & Accessories       7282   434478621   
112  Safety Wash Station Parts & Accessories       7282   434482096   
113  Safety Wash Station Parts & Accessories       7282   434482097   
114  Safety Wash Station Parts & Accessories       7282   434482102   
117  Safety Wash Station Parts & Accessories       7282   434510128   
118  Safety Wash Station Parts & Accessories       7282   434510129   
119  Safety Wash Station Parts & Accessories       7282   434510130   
146               Paint, Stains, & Varnishes       6778   434490442   
147               Paint, Stains, & Varnishes       6778   434490443   
148               Paint, Stains, & Varnishes       6778   434494988   
149               Paint, Stains, & Varnishes       6778   434494989   
150               Paint, Stains, & Varnishes       6778   434494990   
151               Paint, Stains, & Varnishes       6778   434494991   
152               Paint, Stains, & Varnishes       6778   434494992   
153               Paint, Stains, & Varnishes       6778   434494993   
154               Paint, Stains, & Varnishes       6778   434494994   
155               Paint, Stains, & Varnishes       6778   434494995   
156               Paint, Stains, & Varnishes       6778   434494996   
157               Paint, Stains, & Varnishes       6778   434495003   
207                     Safety Wash Stations       7281   434482098   
208                     Safety Wash Stations       7281   434482099   

    external_id  \
31   5637240819   
32   5637240820   
33   5637240487   
34   5637240488   
35   5637239684   
41   5637308028   
42   5637303199   
43   5637303200   
51   5637382504   
52   5637382505   
94   5637166345   
96   5637153548   
97   5637153549   
98   5637153560   
111  5637145381   
112  5637155759   
113  5637155760   
114  5637155767   
117  5637487731   
118  5637487732   
119  5637487733   
146  5637466690   
147  5637466691   
148  5637467450   
149  5637467451   
150  5637467452   
151  5637467453   
152  5637467454   
153  5637467455   
154  5637467456   
155  5637467457   
156  5637467458   
157  5637467465   
207  5637155761   
208  5637155762   

                                                                                                  product_name  \
31                                                         Elmer's Washable School Glue Sticks, Purple, 30/Box   
32                                                                 Elmer's Washable School Glue Sticks, 30/Box   
33                                                        Loctite Super Glue Bottle, .18 oz, Super Glue Liquid   
34                                              Loctite Super Glue Easy Squeeze Gel, .14 oz, Super Glue Liquid   
35                                                          Crayola .29 oz Washable Glue Sticks, Bulk Pack, DZ   
41                                                                   Elmer's Washable School Glue Stick, 12/BX 

In [18]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcust))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchoz))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

matchcust
matchoz
matchesna


In [ ]:
stop

# send to the folder for upload

In [21]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchoz)

In [22]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchcust)

In [23]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesna)